In [24]:
# import libs
import torch
import torchaudio
import os
import numpy as np
import random
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["USER"] = "YOUR_USERNAME" # TODO change this to your username

from data.tokenizer import (
    AudioTokenizer,
    TextTokenizer,
)
from inference_speech_editing_scale import get_mask_interval, inference_one_sample
from edit_utils import get_edits, parse_edit

from argparse import Namespace
from models import voicecraft

In [3]:
# install MFA models and dictionaries if you haven't done so already
# !source ~/.bashrc && \
#     conda activate voicecraft && \
#     mfa model download dictionary english_us_arpa && \
#     mfa model download acoustic english_us_arpa

In [ ]:
# hyperparameters for inference
left_margin = 0.08
right_margin = 0.08
sub_amount = 0.01
codec_audio_sr = 16000
codec_sr = 50
top_k = 40
top_p = 1
temperature = 1
kvcache = 0
# adjust the below three arguments if the generation is not as good
seed = 1 # random seed magic
silence_tokens = [1388,1898,131] # if there are long silence in the generated audio, reduce the stop_repetition to 3, 2 or even 1
stop_repetition = -1 # -1 means do not adjust prob of silence tokens. if there are long silence or unnaturally strecthed words, increase sample_batch_size to 2, 3 or even 4
# what this will do to the model is that the model will run sample_batch_size examples of the same audio, and pick the one that's the shortest
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_everything(seed)
device = "cuda" if torch.cuda.is_available() else "cpu"
# load model, tokenizer, and other necessary files
voicecraft_name="giga330M.pth" # or gigaHalfLibri330M_TTSEnhanced_max16s.pth, giga830M.pth

# the new way of loading the model, with huggingface, recommended
model = voicecraft.VoiceCraft.from_pretrained(f"pyp1/VoiceCraft_{voicecraft_name.replace('.pth', '')}")
phn2num = model.args.phn2num
config = vars(model.args)
model.to(device)

# # the old way of loading the model
# from models import voicecraft
# filepath = hf_hub_download(repo_id="pyp1/VoiceCraft", filename=voicecraft_name, repo_type="model")
# ckpt = torch.load(filepath, map_location="cpu")
# model = voicecraft.VoiceCraft(ckpt["config"])
# model.load_state_dict(ckpt["model"])
# config = vars(model.args)
# phn2num = ckpt["phn2num"]
# model.to(device)
# model.eval()

encodec_fn = "./pretrained_models/encodec_4cb2048_giga.th"
if not os.path.exists(encodec_fn):
    os.system(f"wget https://huggingface.co/pyp1/VoiceCraft/resolve/main/encodec_4cb2048_giga.th")
    os.system(f"mv encodec_4cb2048_giga.th ./pretrained_models/encodec_4cb2048_giga.th")
audio_tokenizer = AudioTokenizer(signature=encodec_fn) # will also put the neural codec model on gpu

text_tokenizer = TextTokenizer(backend="espeak")

# point to the original file or record the file
# write down the transcript for the file, or run whisper to get the transcript (and you can modify it if it's not accurate), save it as a .txt file
orig_audio = "./demo/84_121550_000074_000000.wav"
orig_transcript = "But when I had approached so near to them The common object, which the sense deceives, Lost not by distance any of its marks,"
# move the audio and transcript to temp folder
temp_folder = "./demo/temp"
os.makedirs(temp_folder, exist_ok=True)
os.system(f"cp {orig_audio} {temp_folder}")
filename = os.path.splitext(orig_audio.split("/")[-1])[0]
with open(f"{temp_folder}/{filename}.txt", "w") as f:
    f.write(orig_transcript)
# run MFA to get the alignment
align_temp = f"{temp_folder}/mfa_alignments"
os.makedirs(align_temp, exist_ok=True)
os.system(f"mfa align -j 1 --output_format csv {temp_folder} english_us_arpa english_us_arpa {align_temp}")
# if it fail, it could be because the audio is too hard for the alignment model, increasing the beam size usually solves the issue
# os.system(f"mfa align -j 1 --output_format csv {temp_folder} english_us_arpa english_us_arpa {align_temp} --beam 1000 --retry_beam 2000")
audio_fn = f"{temp_folder}/{filename}.wav"
transcript_fn = f"{temp_folder}/{filename}.txt"
align_fn = f"{align_temp}/{filename}.csv"



In [ ]:
# propose what do you want the target modified transcript to be
target_transcript = "But when I had approached so near, that the sense deceives, Lost not by farness any of its marks,"
print("orig: ", orig_transcript)
print("trgt: ", target_transcript)

# run the script to turn user input to the format that the model can take
operations, orig_span, new_span = parse_edit(orig_transcript, target_transcript)
if operations[-1] == 'i':
    raise RuntimeError("The last operation should not be insertion. Please use text to speech instead")
print(operations)
used_edits = get_edits(operations)
print(used_edits)

def process_span(span):
    if span[0] > span[1]:
        raise RuntimeError(f"example {audio_fn} failed")
    if span[0] == span[1]:
        return [span[0]]
    return span

print("orig_span: ", orig_span)
print("new_span: ", new_span)
orig_span_save = [process_span(span) for span in orig_span]
new_span_save = [process_span(span) for span in new_span]

orig_span_saves = [",".join([str(item) for item in span]) for span in orig_span_save]
new_span_saves = [",".join([str(item) for item in span]) for span in new_span_save]

starting_intervals = []
ending_intervals = []
for i, orig_span_save in enumerate(orig_span_saves):
  start, end = get_mask_interval(align_fn, orig_span_save, used_edits[i])
  starting_intervals.append(start)
  ending_intervals.append(end)

print("intervals: ", starting_intervals, ending_intervals)

info = torchaudio.info(audio_fn)
audio_dur = info.num_frames / info.sample_rate

def resolve_overlap(starting_intervals, ending_intervals, audio_dur, codec_sr, left_margin, right_margin, sub_amount):
    while True:
        morphed_span = [(max(start - left_margin, 1/codec_sr), min(end + right_margin, audio_dur))
                        for start, end in zip(starting_intervals, ending_intervals)] # in seconds
        mask_interval = [[round(span[0]*codec_sr), round(span[1]*codec_sr)] for span in morphed_span]
        # Check for overlap
        overlapping = any(a[1] >= b[0] for a, b in zip(mask_interval, mask_interval[1:]))
        if not overlapping:
            break
        
        # Reduce margins
        left_margin -= sub_amount
        right_margin -= sub_amount
    
    return mask_interval


# span in codec frames
mask_interval = resolve_overlap(starting_intervals, ending_intervals, audio_dur, codec_sr, left_margin, right_margin, sub_amount)
mask_interval = torch.LongTensor(mask_interval) # [M,2], M==1 for now
# run the model to get the output
decode_config = {'top_k': top_k, 'top_p': top_p, 'temperature': temperature, 'stop_repetition': stop_repetition, 'kvcache': kvcache, "codec_audio_sr": codec_audio_sr, "codec_sr": codec_sr, "silence_tokens": silence_tokens}
orig_audio, new_audio = inference_one_sample(model, Namespace(**config), phn2num, text_tokenizer, audio_tokenizer, audio_fn, target_transcript, mask_interval, device, decode_config)

# save segments for comparison
orig_audio, new_audio = orig_audio[0].cpu(), new_audio[0].cpu()
# logging.info(f"length of the resynthesize orig audio: {orig_audio.shape}")

# display the audio
from IPython.display import Audio
print("original:")
display(Audio(orig_audio, rate=codec_audio_sr))

print("edited:")
display(Audio(new_audio, rate=codec_audio_sr))

# # save the audio
# # output_dir
# output_dir = "./demo/generated_se"
# os.makedirs(output_dir, exist_ok=True)

# save_fn_new = f"{output_dir}/{os.path.basename(audio_fn)[:-4]}_new_seed{seed}.wav"

# torchaudio.save(save_fn_new, new_audio, codec_audio_sr)

# save_fn_orig = f"{output_dir}/{os.path.basename(audio_fn)[:-4]}_orig.wav"
# if not os.path.isfile(save_fn_orig):
#     orig_audio, orig_sr = torchaudio.load(audio_fn)
#     if orig_sr != codec_audio_sr:
#         orig_audio = torchaudio.transforms.Resample(orig_sr, codec_audio_sr)(orig_audio)
#     torchaudio.save(save_fn_orig, orig_audio, codec_audio_sr)

# # if you get error importing T5 in transformers
# # try
# # pip uninstall Pillow
# # pip install Pillow
# # you are likely to get warning looks like WARNING:phonemizer:words count mismatch on 300.0% of the lines (3/1), this can be safely ignored